# Azure OpenAI Assistants

Azure OpenAI Assistants allows you to create AI assistants tailored to your needs through custom instructions and augmented by advanced tools like code interpreter, and custom functions. In this article we'll provide an in-depth walkthrough of getting started with the Assistants API.

In [2]:
%pip install openai --upgrade
%pip install tiktoken
%pip install numpy
%pip install python-dotenv
%pip install pillow
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv("azure.env")

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version=os.getenv("API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model=os.getenv("MODEL_VERSION") #You must replace this value with the deployment name of your model.
)

In [4]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_DoLK3PZc4r4aZwAroDr2PMaw",
  "created_at": 1721284227,
  "description": null,
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4-2024-04-09",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "response_format": "auto",
  "temperature": 1.0,
  "tool_resources": null,
  "top_p": 1.0,
  "file_ids": []
}


In [5]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_KMVzowEAYVbHAxmuQk5paHpF', created_at=1721284237, metadata={}, object='thread', tool_resources=None)


In [6]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

In [7]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_Tm9ZM2cPDQzlHy7In18uyXqb",
      "assistant_id": null,
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1721284240,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_KMVzowEAYVbHAxmuQk5paHpF",
      "file_ids": []
    }
  ],
  "object": "list",
  "first_id": "msg_Tm9ZM2cPDQzlHy7In18uyXqb",
  "last_id": "msg_Tm9ZM2cPDQzlHy7In18uyXqb",
  "has_more": false
}


We could also pass an instructions parameter here, but this would override the existing instructions that we have already provided for the assistant.

In [8]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [12]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

completed


In [13]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_CAHONtJkAbcBnH6e2Pdqihps",
      "assistant_id": "asst_DoLK3PZc4r4aZwAroDr2PMaw",
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-0PruTyPFM7GQ7n9idGJbcqut",
            "detail": null
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the visualization of the sine wave for \\( sin(x) \\) from \\( x = 0 \\) to \\( x = 2\\pi \\). If you need any adjustments or additional information on the graph, feel free to let me know!"
          },
          "type": "text"
        }
      ],
      "created_at": 1721284267,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_ZFs7ejMtSuh3pSKE72KU07KK",
      "status": null,
      "thread_id": "thread_KMVzowEAYVbHAxm

In [14]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(data)  

{'data': [{'id': 'msg_CAHONtJkAbcBnH6e2Pdqihps', 'assistant_id': 'asst_DoLK3PZc4r4aZwAroDr2PMaw', 'attachments': None, 'completed_at': None, 'content': [{'image_file': {'file_id': 'assistant-0PruTyPFM7GQ7n9idGJbcqut', 'detail': None}, 'type': 'image_file'}, {'text': {'annotations': [], 'value': 'Here is the visualization of the sine wave for \\( sin(x) \\) from \\( x = 0 \\) to \\( x = 2\\pi \\). If you need any adjustments or additional information on the graph, feel free to let me know!'}, 'type': 'text'}], 'created_at': 1721284267, 'incomplete_at': None, 'incomplete_details': None, 'metadata': {}, 'object': 'thread.message', 'role': 'assistant', 'run_id': 'run_ZFs7ejMtSuh3pSKE72KU07KK', 'status': None, 'thread_id': 'thread_KMVzowEAYVbHAxmuQk5paHpF', 'file_ids': []}, {'id': 'msg_DYGCTLnUsPdqueRCSPTT7YP2', 'assistant_id': 'asst_DoLK3PZc4r4aZwAroDr2PMaw', 'attachments': None, 'completed_at': None, 'content': [{'text': {'annotations': [], 'value': "To create a visualization of a sine wa

In [15]:
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

In [16]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("sinewave.png")
image.show()

In [17]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the sinewave"
)

In [18]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)


In [22]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
status = run.status
print(status)

completed


* Please wait until you get "completed"

In [23]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_X0LXudZnaQ9zHWdQwUgpjhxL",
      "assistant_id": "asst_DoLK3PZc4r4aZwAroDr2PMaw",
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Certainly! Here is the code I used to generate and plot the sine wave:\n\n```python\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Generating data\nx = np.linspace(0, 2 * np.pi, 100)  # 100 points from 0 to 2*pi\ny = np.sin(x)  # sine values\n\n# Plotting the sine wave\nplt.figure(figsize=(10, 5))\nplt.plot(x, y, label='sin(x)')\nplt.title('Sine Wave')\nplt.xlabel('x')\nplt.ylabel('sin(x)')\nplt.grid(True)\nplt.legend()\nplt.show()\n```\n\nThis code uses NumPy to create an array of 100 points linearly spaced between 0 and \\(2\\pi\\) and then computes the sine of each value. The results are plotted using Matplotlib, which is a popular plotting library in Python. The plot is configured to be 10x5 inches, 

In [24]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

Certainly! Here is the code I used to generate and plot the sine wave:

```python
import numpy as np
import matplotlib.pyplot as plt

# Generating data
x = np.linspace(0, 2 * np.pi, 100)  # 100 points from 0 to 2*pi
y = np.sin(x)  # sine values

# Plotting the sine wave
plt.figure(figsize=(10, 5))
plt.plot(x, y, label='sin(x)')
plt.title('Sine Wave')
plt.xlabel('x')
plt.ylabel('sin(x)')
plt.grid(True)
plt.legend()
plt.show()
```

This code uses NumPy to create an array of 100 points linearly spaced between 0 and \(2\pi\) and then computes the sine of each value. The results are plotted using Matplotlib, which is a popular plotting library in Python. The plot is configured to be 10x5 inches, include grid lines, labels, and a legend for clarity.


In [25]:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_DoLK3PZc4r4aZwAroDr2PMaw', deleted=True, object='assistant.deleted')